## 분석 목표 : Favorita stores 에서 판매되는 수천개의 제품군의 판매량을 예측하기. 

'Corporación Favorita'라는 에콰도르의 대형 식료품 소매업체 : 남미의 다른 국가(7개국)에도 진출해있음. https://www.corporacionfavorita.com/en/


데이터에는 **54개의 store, 33개의 제품군**이 존재.

날짜는 ***2013-01-01 부터 2017-08-31***까지. (테스트데이터의 날짜는 train data의 마지막 날짜 이후 15일)

*main mission : 각 제품군과 매장 조합의 매출을 예측하기. *



#### **데이터 분석을 위한 배경 지식**
- 에콰도르는 석유 의존도가 높은 나라이다. 따라서 유가에 따라 경제가 영향을 많이 받는다. 어떤 제품군이 유가에 의해 긍정적/부정적 영향을 받았는지 daily oil price 데이터로 알 수 있을 것.
- 공공부문 임금은 2주 간격으로 15일과 월말에 지급되는데, 슈퍼마켓 매출이 영향을 받을 수 있습니다.
- 2016년 4월 16일 에콰도르에서 규모 7.8의 지진이 발생했습니다. 사람들은 지진 후 몇 주 동안 슈퍼마켓 판매에 큰 영향을 미친 물과 기타 우선 필요한 제품을 기부하는 구호 활동에 동참했습니다.


store_nbr은 제품이 판매되는 매장을 식별합니다. 

family는 판매되는 제품 유형을 식별합니다. 

sales는 특정 날짜에 특정 매장에서 제품군에 대한 총 판매량을 제공합니다. 제품이 분수 단위(예: 칩 1봉지가 아닌 치즈 1.5kg)로 판매될 수 있으므로 분수 값이 가능합니다. 

onpromotion은 특정 날짜에 매장에서 판촉되었던 제품군의 총 항목 수를 제공합니다.

In [61]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tableau-transactions/.png
/kaggle/input/tableau-transactions/store sale -    .png
/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


## Importing data

In [19]:
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')

train

id        date  store_nbr                      family     sales  \
0              0  2013-01-01          1                  AUTOMOTIVE     0.000   
1              1  2013-01-01          1                   BABY CARE     0.000   
2              2  2013-01-01          1                      BEAUTY     0.000   
3              3  2013-01-01          1                   BEVERAGES     0.000   
4              4  2013-01-01          1                       BOOKS     0.000   
...          ...         ...        ...                         ...       ...   
3000883  3000883  2017-08-15          9                     POULTRY   438.133   
3000884  3000884  2017-08-15          9              PREPARED FOODS   154.553   
3000885  3000885  2017-08-15          9                     PRODUCE  2419.729   
3000886  3000886  2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3000887  3000887  2017-08-15          9                     SEAFOOD    16.000   

         onpromotion  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  
...              ...  
3000883            0  
3000884            1  
3000885          148  
3000886            8  
3000887            0  

[3000888 rows x 6 columns]

In [5]:
train['family'].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [17]:
transaction = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')
transaction.sort_values(['store_nbr', 'date'])

date  store_nbr  transactions
1      2013-01-02          1          2111
47     2013-01-03          1          1833
93     2013-01-04          1          1863
139    2013-01-05          1          1509
185    2013-01-06          1           520
...           ...        ...           ...
83271  2017-08-11         54           768
83325  2017-08-12         54           903
83379  2017-08-13         54          1054
83433  2017-08-14         54           818
83487  2017-08-15         54           802

[83488 rows x 3 columns]

In [22]:
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")
stores

store_nbr           city                           state type  cluster
0           1          Quito                       Pichincha    D       13
1           2          Quito                       Pichincha    D       13
2           3          Quito                       Pichincha    D        8
3           4          Quito                       Pichincha    D        9
4           5  Santo Domingo  Santo Domingo de los Tsachilas    D        4
5           6          Quito                       Pichincha    D       13
6           7          Quito                       Pichincha    D        8
7           8          Quito                       Pichincha    D        8
8           9          Quito                       Pichincha    B        6
9          10          Quito                       Pichincha    C       15
10         11        Cayambe                       Pichincha    B        6
11         12      Latacunga                        Cotopaxi    C       15
12         13      Latacunga                        Cotopaxi    C       15
13         14       Riobamba                      Chimborazo    C        7
14         15         Ibarra                        Imbabura    C       15
15         16  Santo Domingo  Santo Domingo de los Tsachilas    C        3
16         17          Quito                       Pichincha    C       12
17         18          Quito                       Pichincha    B       16
18         19       Guaranda                         Bolivar    C       15
19         20          Quito                       Pichincha    B        6
20         21  Santo Domingo  Santo Domingo de los Tsachilas    B        6
21         22           Puyo                         Pastaza    C        7
22         23         Ambato                      Tungurahua    D        9
23         24      Guayaquil                          Guayas    D        1
24         25        Salinas                     Santa Elena    D        1
25         26      Guayaquil                          Guayas    D       10
26         27          Daule                          Guayas    D        1
27         28      Guayaquil                          Guayas    E       10
28         29      Guayaquil                          Guayas    E       10
29         30      Guayaquil                          Guayas    C        3
30         31       Babahoyo                        Los Rios    B       10
31         32      Guayaquil                          Guayas    C        3
32         33        Quevedo                        Los Rios    C        3
33         34      Guayaquil                          Guayas    B        6
34         35         Playas                          Guayas    C        3
35         36       Libertad                          Guayas    E       10
36         37         Cuenca                           Azuay    D        2
37         38           Loja                            Loja    D        4
38         39         Cuenca                           Azuay    B        6
39         40        Machala                          El Oro    C        3
40         41        Machala                          El Oro    D        4
41         42         Cuenca                           Azuay    D        2
42         43     Esmeraldas                      Esmeraldas    E       10
43         44          Quito                       Pichincha    A        5
44         45          Quito                       Pichincha    A       11
45         46          Quito                       Pichincha    A       14
46         47          Quito                       Pichincha    A       14
47         48          Quito                       Pichincha    A       14
48         49          Quito                       Pichincha    A       11
49         50         Ambato                      Tungurahua    A       14
50         51      Guayaquil                          Guayas    A       17
51         52          Manta                          Manabi    A       11
52         53          Manta 

### 날짜 data type 바꿔주기. 

In [21]:
train['date'] = pd.to_datetime(train['date'])
transaction['date'] = pd.to_datetime(transaction['date'])
test['date'] = pd.to_datetime(test['date'])

In [9]:
train.date

0         2013-01-01
1         2013-01-01
2         2013-01-01
3         2013-01-01
4         2013-01-01
             ...    
3000883   2017-08-15
3000884   2017-08-15
3000885   2017-08-15
3000886   2017-08-15
3000887   2017-08-15
Name: date, Length: 3000888, dtype: datetime64[ns]

In [50]:
# 엑셀로 데이터프레임 저장
excel_file_path = 'transaction.xlsx'
transaction.to_excel(excel_file_path, index=False)

excel_file_path = 'train.xlsx'
temp.to_excel(excel_file_path, index=False)


In [33]:
temp = train.groupby(["date", "store_nbr"])
temp = temp['sales'].sum().reset_index()
temp

date  store_nbr         sales
0     2013-01-01          1      0.000000
1     2013-01-01          2      0.000000
2     2013-01-01          3      0.000000
3     2013-01-01          4      0.000000
4     2013-01-01          5      0.000000
...          ...        ...           ...
90931 2017-08-15         50  16879.121004
90932 2017-08-15         51  20154.559000
90933 2017-08-15         52  18600.046000
90934 2017-08-15         53   8208.189000
90935 2017-08-15         54  12666.858000

[90936 rows x 3 columns]

In [34]:
mer1 = pd.merge(temp, transaction, how = "left")
mer1

date  store_nbr         sales  transactions
0     2013-01-01          1      0.000000           NaN
1     2013-01-01          2      0.000000           NaN
2     2013-01-01          3      0.000000           NaN
3     2013-01-01          4      0.000000           NaN
4     2013-01-01          5      0.000000           NaN
...          ...        ...           ...           ...
90931 2017-08-15         50  16879.121004        2804.0
90932 2017-08-15         51  20154.559000        1573.0
90933 2017-08-15         52  18600.046000        2255.0
90934 2017-08-15         53   8208.189000         932.0
90935 2017-08-15         54  12666.858000         802.0

[90936 rows x 4 columns]

In [44]:
#"sales"(판매량)과 "transactions"(거래량)의 상관계수
mer1['sales'].corr(mer1["transactions"])

0.8373836801431722

In [48]:
import plotly.express as px
px.line(transaction.sort_values(["store_nbr", "date"]), x='date', y='transactions', color='store_nbr',title = "Transactions" )

In [71]:
a = transaction.copy()
a["year"] = a['date'].dt.year
a["month"] = a['date'].dt.month
px.box(a, x="year", y="transactions" , color = "month", title = "Transactions")